# Localised Regression charts

## Set-up

In [ ]:
# external imports
from IPython.display import display

# local import
import plotting
from common import ATTITUDINAL, VOTING_INTENTION, ensure
from data_capture import retrieve

In [ ]:
SHOW = True

In [ ]:
## Plotting initialisation
def set_up_plotting() -> None:
    """Set up plotting."""

    p = plotting.get_chart_dir()
    plotting.set_chart_dir(p + '/loess-regression/')
    plotting.clear_chart_dir()


set_up_plotting()

## Get data

In [ ]:
data = retrieve()
ensure(data, "You must run the data capture notebook every day.")

In [ ]:
# Double check that our voting intention data has been normalized ...
# Ideally, this cell will show nothing is wrong.

checkable = "Primary", "2pp"
vi = data[VOTING_INTENTION]
for check in checkable:
    columns = [c for c in vi.columns if check in c and "Other" not in c]
    check_these = vi[columns]
    row_sum = check_these.sum(axis=1)
    fail = check_these[(row_sum > 100.01) | (row_sum < 99.99)].copy()
    if len(fail):
        fail["TOTAL"] = row_sum
        display(fail)

In [ ]:
# Amalgamate the ONP, UAP and OTH columns
vi = plotting.amalgamate_other(data[VOTING_INTENTION])

# let's look at the Primary Vote column names ...
[x for x in vi.columns if "Primary vote" in x]

## And plot

In [ ]:
# Note: sometimes a change in case buggers up the
# plotable selection dictionary in the next code block.
# So let's make sure we have the correct names for each
# column

print(data[ATTITUDINAL].columns)

In [ ]:
# what we will plot ...
plotable = {
    VOTING_INTENTION: {
        # regex patterns for matching desired columns
        r"2pp vote ALP": "Labor 2PP Voting Intention",
        r"2pp vote L/NP": "Coalition 2PP Voting Intention",
        r"Primary vote (L/NP|ALP)": "Major Primary Voting Intention",
        r"Primary vote (GRN|Other)": "minor Primary Voting Intention",
    },
    ATTITUDINAL: {
        # regex patterns for matching desired columns
        r"^Dutton (Satisfied|Dissatisfied)": "Dutton Satisfaction",
        r"^Albanese (Satisfied|Dissatisfied)": "Albanese Satisfaction",
        r"Preferred prime minister (Dutton|Albanese)": "Preferred Prime Minister",
        r"(Dutton|Albanese) Net": "Net Satisfaction",
    },
}

plotting.plot_loess(data, plotable, show=SHOW)

## Finished

In [ ]:
%load_ext watermark
%watermark --python --machine --iversions --watermark

In [ ]:
print("Finished")